# 5. Distance moduli and Hubble residuals

Converts SALT2 fit parameters into **observed distance modulus** and compares to a fiducial cosmology.

- **Redshift:** Prefer TNS host redshift (`tns_redshift`) from `ztf_cleansed.csv` when available, else BTS `redshift`; fallback to redshift from the fit if missing. Used for sncosmo (notebook 4) and all plots. TNS is also written to the output CSV for reference.
- **Peak B magnitude:** mB from x0 (e.g. mB = −2.5 log₁₀(x0) + 10.635).
- **Tripp standardization:** μ_obs = mB − M + α x1 − β c (fixed M, α, β; e.g. M = −19.36, α = 0.14, β = 3.1).
- **Theoretical distance modulus:** μ_th from FlatLambdaCDM(H0=70, Om0=0.3).
- **Hubble residual:** resid = μ_obs − μ_th.

**Output:** `runs/<run>/distance_process.csv` - ztf_id, redshift, tns_redshift (reference), SALT2 params, mB, mu_obs, mu_th, resid.

In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import sncosmo
from astropy.cosmology import FlatLambdaCDM

project_root = Path.cwd().parent
print(f"Project root: {project_root}")

Project root: /Users/david/Code/msc


In [5]:
# User input: run folder name 
folder_name = input("Enter the run folder name: ").strip()
run_folder = project_root / "runs" / folder_name

params_path = run_folder / "sncosmo_fits.csv"

df = pd.read_csv(params_path)
print(f"Loaded {len(df)} object(s) from {params_path}")
df.head()

# c : colour paramter, + means redder
# x1 : stretch parameter, + means broader lightcurve
# redshift : calculated from sncosmo fit
# t0 : time of maximum brightness in B-band
# x0 : overall flux scaling parameter
# x1 : stretch parameter, + means broader lightcurve

Loaded 1275 object(s) from /Users/david/Code/msc/runs/run2/sncosmo_fits.csv


,ztf_id,redshift,ncall,ndof,chisq,t0,x0,x1,c,fit_numerical_warning,fit
0,ZTF17aadlxmv,0.062000,113,22,16.745221,58878.441097,0.001051,0.782320,0.125209,False,0.761146
1,ZTF18aaaqexr,0.070200,101,7,3.710129,58893.649163,0.000589,-1.246107,0.076504,False,0.530018
2,ZTF18aaeqjmc,0.043000,151,8,9.818289,60266.905510,0.002637,-0.664076,0.070290,False,1.227286
3,ZTF18aafdigb,0.063000,103,10,8.609961,60231.409033,0.000684,-2.905661,0.054759,False,0.860996
4,ZTF18aagkwgz,0.037759,113,20,16.020661,60583.391664,0.002406,0.430958,0.146203,False,0.801033


In [6]:
import numpy as np
import pandas as pd
from astropy.cosmology import FlatLambdaCDM


# Fixed nuisance parameters,  depend on host galaxy, https://www.aanda.org/articles/aa/pdf/2014/08/aa23413-14.pdf
# mu = mB - M + alpha*x1 - beta*c
alpha = 0.14
beta  = 3.1
M     = -19.36

# cosmology for theoretical distance modulus
cosmo = FlatLambdaCDM(H0=70.0, Om0=0.3)

ztf_cleansed_path = project_root / "ztf_cleansed.csv"
ztf_df = pd.read_csv(ztf_cleansed_path)

# Prefer TNS host redshift for cosmology and plots; fallback to BTS redshift
z_col = ztf_df["tns_redshift"].fillna(ztf_df["redshift"]) if "tns_redshift" in ztf_df.columns else ztf_df["redshift"]
hostz_map = dict(zip(ztf_df["ZTFID"].astype(str), z_col))
tnsz_map = dict(zip(ztf_df["ZTFID"].astype(str), ztf_df["tns_redshift"])) if "tns_redshift" in ztf_df.columns else {}

processed_rows = []

for _, row in df.iterrows():
    ztf_id = str(row["ztf_id"])

    # Prefer redshift from the fit CSV (full precision from notebook 4); fallback to cleansed catalog
    z = row.get("redshift", np.nan)
    if pd.isna(z) or (isinstance(z, str) and not str(z).strip()):
        z = hostz_map.get(ztf_id, np.nan)
    if pd.isna(z) or (isinstance(z, str) and not str(z).strip()):
        print(f"{ztf_id}: missing redshift, skipping.")
        continue
    z = float(z)

    # Read fit parameters
    ncall = int(row["ncall"])
    ndof  = int(row["ndof"])
    chisq = float(row["chisq"])

    t0 = float(row["t0"])
    x0 = float(row["x0"])
    x1 = float(row["x1"])
    c  = float(row["c"])

    if x0 <= 0 or not np.isfinite(x0):
        print(f"{ztf_id}: invalid x0={x0}, skipping.")
        continue

    mB = -2.5 * np.log10(x0) + 10.635  # B peak mag

    # Observed distance modulus after standardization
    mu_obs = mB - M + alpha * x1 - beta * c

    # Theoretical distance modulus cosmology
    mu_th = float(cosmo.distmod(z).value)

    # Hubble residual
    resid = mu_obs - mu_th

    tns_z = tnsz_map.get(ztf_id, np.nan)
    if pd.notna(tns_z):
        tns_z = float(tns_z)
    processed_rows.append({
        "ztf_id": ztf_id,
        "redshift": z,
        "tns_redshift": tns_z,
        "ncall": ncall,
        "ndof": ndof,
        "chisq": chisq,
        "t0": t0,
        "x0": x0,
        "x1": x1,
        "c": c,
        "mB": mB,
        "mu_obs": mu_obs,
        "mu_th": mu_th,
        "resid": resid,
    })

    print(
        f"{ztf_id}: z={z:.5f}, mB={mB:.3f}, mu_obs={mu_obs:.3f}, "
        f"mu_th={mu_th:.3f}, resid={resid:.3f}"
    )

# Save results
output_df = pd.DataFrame(processed_rows)
output_path = run_folder / "distance_process.csv"
output_df.to_csv(output_path, index=False)
print(f"\nWrote processed results to {output_path}")


ZTF17aadlxmv: z=0.06200, mB=18.081, mu_obs=37.162, mu_th=37.220, resid=-0.058
ZTF18aaaqexr: z=0.07020, mB=18.710, mu_obs=37.658, mu_th=37.503, resid=0.155
ZTF18aaeqjmc: z=0.04300, mB=17.082, mu_obs=36.131, mu_th=36.396, resid=-0.265
ZTF18aafdigb: z=0.06300, mB=18.548, mu_obs=37.331, mu_th=37.257, resid=0.075
ZTF18aagkwgz: z=0.03776, mB=17.182, mu_obs=36.149, mu_th=36.106, resid=0.043
ZTF18aagtwyh: z=0.06604, mB=18.356, mu_obs=37.577, mu_th=37.364, resid=0.213
ZTF18aahtjsc: z=0.05290, mB=18.623, mu_obs=37.091, mu_th=36.862, resid=0.229
ZTF18aaisqmw: z=0.05270, mB=18.335, mu_obs=37.378, mu_th=36.853, resid=0.525
ZTF18aaizerg: z=0.06840, mB=18.585, mu_obs=37.478, mu_th=37.443, resid=0.035
ZTF18aakitiq: z=0.03671, mB=17.519, mu_obs=36.313, mu_th=36.043, resid=0.271
ZTF18aaklpdo: z=0.04600, mB=17.312, mu_obs=36.693, mu_th=36.547, resid=0.146
ZTF18aamvfeb: z=0.03149, mB=16.699, mu_obs=35.804, mu_th=35.701, resid=0.103
ZTF18aamzgzi: z=0.01790, mB=15.152, mu_obs=34.726, mu_th=34.453, resid=0.2